In [1]:
import pandas as pd
import requests as req

In [2]:
URV = 'https://rate.bot.com.tw/xrt?Lang=zh-TW'

s = req.get(URV)

df = pd.read_html(s.text)

In [3]:
columns = pd.MultiIndex.from_product([('現金匯率','即期匯率'),('本行買入','本行賣出')])

In [4]:
import numpy as np

In [5]:
L = [i[:i.index(')')+1] for i in df[0][('幣別','幣別')].values]
df_coin = pd.DataFrame(df[0].iloc[:,1:5].values,columns = columns,index = L)
for i in range(len(df_coin.columns)):
    for j in range(len(df_coin.index)):
        if df_coin.iloc[j,i]!='-':
            df_coin.iloc[j,i] = float(df_coin.iloc[j,i])
        else:
            df_coin.iloc[j,i] = 1000000

In [7]:
import math
df_coin['即期點差'] = (df_coin.iloc[:,3]-df_coin.iloc[:,2])/df_coin.iloc[:,3]
df_coin['即期點差'] = np.array([round(i,3) for i in df_coin['即期點差']])
df_coin

現金匯率              即期匯率            即期點差
               本行買入     本行賣出     本行買入     本行賣出       
美金 (USD)      29.44    30.11    29.79    29.89  0.003
港幣 (HKD)      3.693    3.897    3.819    3.879  0.015
英鎊 (GBP)      35.76    37.88    36.76    37.18  0.011
澳幣 (AUD)       19.1    19.88    19.37     19.6  0.012
加拿大幣 (CAD)    20.92    21.83    21.31    21.53  0.010
新加坡幣 (SGD)    20.53    21.44    21.02     21.2  0.008
瑞士法郎 (CHF)    29.89    31.09    30.55    30.84  0.009
日圓 (JPY)     0.2692    0.282   0.2765   0.2805  0.014
南非幣 (ZAR)   1000000  1000000    1.594    1.674  0.048
瑞典幣 (SEK)      2.66     3.18        3      3.1  0.032
紐元 (NZD)      17.78    18.63    18.16    18.36  0.011
泰幣 (THB)     0.8013   0.9913   0.9137   0.9537  0.042
菲國比索 (PHP)   0.5199   0.6529  1000000  1000000  0.000
印尼幣 (IDR)   0.00168  0.00238  1000000  1000000  0.000
歐元 (EUR)      31.52    32.86    32.14    32.54  0.012
韓元 (KRW)    0.02274  0.02664  1000000  1000000  0.000
越南盾 (VND)    0.0009   0.0014  1000000  1000000  0.000
馬來幣 (MYR)     5.774    7.399  1000000  1000000  0.000
人民幣 (CNY)      4.11    4.272    4.182    4.232  0.012

In [8]:
URL_2 = 'https://rate.bot.com.tw/ir?Lang=zh-TW'
s_rate = req.get(URL_2)
df_rate = pd.read_html(s_rate.text)

In [17]:
df_rate[0].iloc[:,-3]

0     0.700
1     0.750
2     0.550
3     0.200
4     0.650
5     0.500
6     0.300
7     0.001
8     0.002
9     3.200
10    0.001
11    0.700
12    0.002
13    1.500
Name: (定期存款 (年息%), 6 個月), dtype: float64

In [25]:
df_rate_Final = pd.DataFrame({'定期年息':df_rate[0].iloc[:,-3].values/100},index=[i[:i.index(')')+1] for i in df_rate[0][('幣別','幣別')]])
df_rate_Final

,定期年息
美金 (USD),0.00700
美金 (USD),0.00750
港幣 (HKD),0.00550
英鎊 (GBP),0.00200
澳幣 (AUD),0.00650
加拿大幣 (CAD),0.00500
新加坡幣 (SGD),0.00300
瑞士法郎 (CHF),0.00001
日圓 (JPY),0.00002
南非幣 (ZAR),0.03200


In [26]:
df_rate_Final = df_rate_Final.iloc[[i!=1 for i in range(len(df_rate_Final.index))],:]

In [27]:
df_rate_Final

,定期年息
美金 (USD),0.00700
港幣 (HKD),0.00550
英鎊 (GBP),0.00200
澳幣 (AUD),0.00650
加拿大幣 (CAD),0.00500
新加坡幣 (SGD),0.00300
瑞士法郎 (CHF),0.00001
日圓 (JPY),0.00002
南非幣 (ZAR),0.03200
瑞典幣 (SEK),0.00001


In [40]:
df_coin['定期年息'] = df_rate_Final.iloc[:,0]

In [47]:
df_coin['獲益率(%)'] = (df_coin['定期年息']-df_coin['即期點差'])*100
df_coin.drop(columns=['獲益率'])

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\generic.py:3812: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


現金匯率              即期匯率            即期點差     定期年息 獲益率(%)
               本行買入     本行賣出     本行買入     本行賣出                       
美金 (USD)      29.44    30.11    29.79    29.89  0.003  0.00700  0.400
港幣 (HKD)      3.693    3.897    3.819    3.879  0.015  0.00550 -0.950
英鎊 (GBP)      35.76    37.88    36.76    37.18  0.011  0.00200 -0.900
澳幣 (AUD)       19.1    19.88    19.37     19.6  0.012  0.00650 -0.550
加拿大幣 (CAD)    20.92    21.83    21.31    21.53  0.010  0.00500 -0.500
新加坡幣 (SGD)    20.53    21.44    21.02     21.2  0.008  0.00300 -0.500
瑞士法郎 (CHF)    29.89    31.09    30.55    30.84  0.009  0.00001 -0.899
日圓 (JPY)     0.2692    0.282   0.2765   0.2805  0.014  0.00002 -1.398
南非幣 (ZAR)   1000000  1000000    1.594    1.674  0.048  0.03200 -1.600
瑞典幣 (SEK)      2.66     3.18        3      3.1  0.032  0.00001 -3.199
紐元 (NZD)      17.78    18.63    18.16    18.36  0.011  0.00700 -0.400
泰幣 (THB)     0.8013   0.9913   0.9137   0.9537  0.042      NaN    NaN
菲國比索 (PHP)   0.5199   0.6529  1000000  1000000  0.000      NaN    NaN
印尼幣 (IDR)   0.00168  0.00238  1000000  1000000  0.000      NaN    NaN
歐元 (EUR)      31.52    32.86    32.14    32.54  0.012  0.00002 -1.198
韓元 (KRW)    0.02274  0.02664  1000000  1000000  0.000      NaN    NaN
越南盾 (VND)    0.0009   0.0014  1000000  1000000  0.000      NaN    NaN
馬來幣 (MYR)     5.774    7.399  1000000  1000000  0.000      NaN    NaN
人民幣 (CNY)      4.11    4.272    4.182    4.232  0.012  0.01500  0.300

In [44]:
df_coin.iloc[:,-1]

美金 (USD)     NaN
港幣 (HKD)     NaN
英鎊 (GBP)     NaN
澳幣 (AUD)     NaN
加拿大幣 (CAD)   NaN
新加坡幣 (SGD)   NaN
瑞士法郎 (CHF)   NaN
日圓 (JPY)     NaN
南非幣 (ZAR)    NaN
瑞典幣 (SEK)    NaN
紐元 (NZD)     NaN
泰幣 (THB)     NaN
菲國比索 (PHP)   NaN
印尼幣 (IDR)    NaN
歐元 (EUR)     NaN
韓元 (KRW)     NaN
越南盾 (VND)    NaN
馬來幣 (MYR)    NaN
人民幣 (CNY)    NaN
Name: (獲益率, ), dtype: float64